### Load parameters

In [103]:
data_main = r'D:\Baylor\Patient12' # data_main is patient level
global N_shrink,N_frame
N_shrink=N_frame = 5

### Load Packages

In [104]:
from PIL import Image
from skimage.util import img_as_float
from skimage import data as dt
import os
import glob
import numpy as np
import pandas as pd
import glob, os.path
from numpy import trapz
import csv

### Define motion image function

In [105]:
def motion_features(imagePath):
    ### Load images
    ### Set directory to image file location
    os.chdir(imagePath)  

    shrink = (slice(0, None, N_shrink), slice(0, None, N_shrink)) # down sample by 10

    file_names=glob.glob('*F855*')[0:400] # load 400 PPG images
    F=[img_as_float(dt.load(imagePath+r'\\'+im_name))[shrink] for im_name in file_names ]

    N_row=F[1].shape[0] # number of rows
    N_col=F[1].shape[1]
    F=np.array(F) # convert from list to 3D image array (400, 209, 282)

    ### Check pixel value range
    data=np.array(F)
    #_=plt.hist(data.flatten())
    ran=data.max()-data.min() # range of data

    ### Calculate frame difference
    #### 5 frames apart
    Diff=[]
    counter=0
    for i in range(0,400-N_frame,N_frame):
        Diff.append(abs(np.array(F[i+N_frame-1]-F[i])).sum())

    # normalize diff by number of pixels per frame, and signal range
    Diff=np.array(Diff)/N_row/N_col/ran
    Diff=Diff-min(Diff)
    
    # Area under the curve
    # Compute the area using the composite trapezoidal rule.
    # http://stackoverflow.com/questions/13320262/calculating-the-area-under-a-curve-given-a-set-of-coordinates-without-knowing-t
    area = trapz(Diff, dx=N_frame)
    
    # maximum motion
    mx = max(Diff)
    
    # mean motion
    mn = np.mean(Diff)
    
    # maximum gradient
    Diff2 = Diff[1:]
    grad = Diff2-Diff[:-1]
    maxgrad = max(abs(grad))
    
    # std of motion
    sd = np.std(Diff)
    
    output_motion = [imagePath,area,mx,mn,maxgrad,sd]
    return output_motion

### Find all images

In [106]:
filesDepth = glob.glob(data_main+'\*\*\*\*')
dirsDepth = filter(lambda f: os.path.isdir(f), filesDepth)

with open(data_main+"\motion.csv", 'wb') as outfile: # make sure file is closed after writing
    writer = csv.writer(outfile)
    writer.writerow(["foldername", "AUC", "max","mean","max frame diff","std"])
    for im in dirsDepth:
        output_motion = motion_features(im)
        writer.writerow(output_motion)